# A/B Testing — A Complete Example

This notebook walks through a full A/B testing workflow:

1. **Problem Setup** — Define hypothesis and metrics
2. **Sample Size Calculation** — Power analysis
3. **Data Simulation** — Generate realistic experiment data
4. **Exploratory Analysis** — Visualize the groups
5. **Statistical Testing** — Z-test for proportions, Chi-squared test, confidence intervals
6. **Bayesian Approach** — Beta-Binomial model as an alternative
7. **Interpretation** — How to make a decision

## 1. Problem Setup

**Scenario:** An e-commerce company wants to test whether a new checkout page design (Variant B) leads to a higher conversion rate than the current design (Variant A).

- **Null Hypothesis ($H_0$):** $p_B = p_A$ — The conversion rates are the same.
- **Alternative Hypothesis ($H_1$):** $p_B \neq p_A$ — The conversion rates differ (two-sided test).
- **Primary Metric:** Conversion rate (proportion of visitors who complete a purchase).
- **Significance Level:** $\alpha = 0.05$
- **Desired Power:** $1 - \beta = 0.80$

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from statsmodels.stats.power import NormalIndPower
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="muted")
np.random.seed(42)

print("All libraries loaded ✓")